In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/investments/sti_return.csv
/kaggle/input/stock-prices/sg_stock_prices.csv


In [2]:
df = pd.read_csv('/kaggle/input/stock-prices/sg_stock_prices.csv')

In [3]:

def cleanMB(text):
    if text=='.':
        text = 0
    return float(text)
df['M/B'] = df['M/B'].apply(cleanMB)

In [4]:
df.head()

,Company Name,GVKEY,Year,Price,Return,Return (t-1),M/B,Size,Score
0,GREAT EASTERN HOLDINGS LTD,15565,2006,17.00,0.156,.,2.74,42.03,4.42
1,GREAT EASTERN HOLDINGS LTD,15565,2007,16.60,-0.024,0.156,2.39,46.52,1.95
2,GREAT EASTERN HOLDINGS LTD,15565,2008,9.06,-0.454,-0.024,1.42,44.03,7.08
3,GREAT EASTERN HOLDINGS LTD,15565,2009,13.54,0.494,-0.454,1.80,48.53,4.16
4,GREAT EASTERN HOLDINGS LTD,15565,2010,15.62,0.154,0.494,1.84,53.37,6.23


In [5]:
def form_portfolios(df, n_groups=5):
    df['size_rank'] = pd.qcut(df['Size'], q=n_groups, labels=False, duplicates='drop') + 1
    df['bm_rank'] = pd.qcut(df['M/B'], q=n_groups, labels=False, duplicates='drop') + 1

    df['portfolio'] = df['size_rank'].astype(str) + 'x' + df['bm_rank'].astype(str)

    portfolio_returns = df.groupby(['Year', 'portfolio'])['Return'].mean().unstack()
    
    return portfolio_returns

portfolio_returns = form_portfolios(df)

portfolio_returns.head()

portfolio,1x1,1x2,1x3,1x4,1x5,2x1,2x2,2x3,2x4,2x5,...,4x1,4x2,4x3,4x4,4x5,5x1,5x2,5x3,5x4,5x5
Year,,,,,,,,,,,,,,,,,,,,,
2006,0.035607,-0.053389,0.196000,0.698920,0.786745,0.116889,0.030474,0.130500,0.252583,0.615893,...,0.151900,0.080000,0.441842,0.303350,0.482885,1.313833,0.463750,0.452778,0.318214,0.469000
2007,0.579900,0.406875,0.403933,0.805516,1.132136,0.525000,0.271909,0.345762,0.576848,0.689342,...,0.286750,0.207150,0.174741,0.256880,0.618800,0.026500,-0.034571,0.217556,0.252387,0.386522
2008,-0.520303,-0.618824,-0.468429,-0.409182,-0.623541,-0.593452,-0.444050,-0.592529,-0.475444,-0.599700,...,-0.636357,-0.567690,-0.487083,-0.581900,-0.477375,-0.651265,-0.596619,-0.470333,-0.508875,-0.231375
2009,0.422444,0.360842,0.376905,0.407864,3.227961,0.142857,0.402593,0.848167,0.774214,1.033719,...,0.679333,0.938583,1.023226,1.080952,0.724696,0.772937,0.902889,0.964778,0.926034,0.662500
2010,-0.273500,-0.014917,0.052452,0.032926,0.106537,0.022100,0.077500,0.520533,0.097174,0.283187,...,0.043565,0.110000,0.133133,0.133476,0.468577,0.190067,0.090889,0.057333,0.142781,0.203389


In [6]:
def calculate_factors(df):
    # Calculate SMB (Small Minus Big)
    small = df.filter(regex='1x|2x').mean(axis=1)  # Average returns of the two smallest size quintiles
    big = df.filter(regex='4x|5x').mean(axis=1)    # Average returns of the two largest size quintiles
    smb = small - big

    # Calculate HML (High Minus Low)
    high_bm = df.filter(regex='x5').mean(axis=1)   # Average returns of the highest B/M quintile
    low_bm = df.filter(regex='x1').mean(axis=1)    # Average returns of the lowest B/M quintile
    hml = high_bm - low_bm

    return smb, hml

smb, hml = calculate_factors(portfolio_returns)

In [7]:
smb

Year
2006   -0.166733
2007    0.334451
2008   -0.013658
2009   -0.067836
2010   -0.066922
2011   -0.044865
2012    0.042415
2013    0.010752
2014   -0.006299
2015    0.003698
2016    0.010454
2017    0.002593
2018   -0.018944
2019   -0.029268
2020    0.085434
2021    0.051700
2022   -0.085991
dtype: float64

In [8]:
market_data = pd.read_csv('/kaggle/input/investments/sti_return.csv')
market_data = market_data[market_data['Year']>=2006]


In [9]:

market_data.head()

,Year,STI,STI Return
6,2006,2985.83,0.27
7,2007,3482.30,0.17
8,2008,1761.56,-0.49
9,2009,2897.62,0.64
10,2010,3190.04,0.10


In [10]:
factors = pd.DataFrame({'smb': smb, 'hml': hml})
factors = factors.merge(market_data, left_index=True, right_on='Year', how='left')
factors['market_excess'] = factors['STI Return'] - 0  

In [11]:
factors.head()


,smb,hml,Year,STI,STI Return,market_excess
6,-0.166733,0.221285,2006,2985.83,0.27,0.27
7,0.334451,0.264053,2007,3482.30,0.17,0.17
8,-0.013658,0.086266,2008,1761.56,-0.49,-0.49
9,-0.067836,0.959928,2009,2897.62,0.64,0.64
10,-0.066922,0.222187,2010,3190.04,0.10,0.10


In [12]:
factors_cleaned = factors[['Year','smb','hml','market_excess']]

In [13]:
factors_cleaned

,Year,smb,hml,market_excess
6,2006,-0.166733,0.221285,0.27
7,2007,0.334451,0.264053,0.17
8,2008,-0.013658,0.086266,-0.49
9,2009,-0.067836,0.959928,0.64
10,2010,-0.066922,0.222187,0.10
11,2011,-0.044865,0.137002,-0.17
12,2012,0.042415,0.630474,0.20
13,2013,0.010752,0.720292,0.00
14,2014,-0.006299,-0.068327,0.06
15,2015,0.003698,0.198938,-0.14


In [14]:
portfolio_returns.shape

(17, 25)

In [15]:
factors.shape

(17, 6)

In [16]:
import statsmodels.api as sm
factors.reset_index(drop=True, inplace=True)


factors.index = portfolio_returns.index

def run_regressions(portfolio_returns, factors):
    results = {}
    for portfolio in portfolio_returns.columns:
        print("Hi")
        y = portfolio_returns[portfolio]  
        X = factors[['market_excess', 'smb', 'hml']]
        X = sm.add_constant(X) 
        model = sm.OLS(y, X, missing='drop').fit()
        results[portfolio] = model.params
        print(model.summary())
    return pd.DataFrame(results)

# Run regression analysis
regression_results = run_regressions(portfolio_returns, factors)
print(regression_results)

Hi
                            OLS Regression Results                            
Dep. Variable:                    1x1   R-squared:                       0.633
Model:                            OLS   Adj. R-squared:                  0.548
Method:                 Least Squares   F-statistic:                     7.465
Date:                Tue, 16 Apr 2024   Prob (F-statistic):            0.00373
Time:                        10:15:17   Log-Likelihood:                 2.5761
No. Observations:                  17   AIC:                             2.848
Df Residuals:                      13   BIC:                             6.181
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -0.0294      0.095     -0.

/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=1

                            OLS Regression Results                            
Dep. Variable:                    3x2   R-squared:                       0.778
Model:                            OLS   Adj. R-squared:                  0.727
Method:                 Least Squares   F-statistic:                     15.17
Date:                Tue, 16 Apr 2024   Prob (F-statistic):           0.000155
Time:                        10:15:17   Log-Likelihood:                 12.204
No. Observations:                  17   AIC:                            -16.41
Df Residuals:                      13   BIC:                            -13.07
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -0.0791      0.054     -1.475

/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=1

                            OLS Regression Results                            
Dep. Variable:                    5x5   R-squared:                       0.840
Model:                            OLS   Adj. R-squared:                  0.803
Method:                 Least Squares   F-statistic:                     22.78
Date:                Tue, 16 Apr 2024   Prob (F-statistic):           1.88e-05
Time:                        10:15:17   Log-Likelihood:                 16.535
No. Observations:                  17   AIC:                            -25.07
Df Residuals:                      13   BIC:                            -21.74
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.1159      0.042      2.787

In [17]:
regression_results

,1x1,1x2,1x3,1x4,1x5,2x1,2x2,2x3,2x4,2x5,...,4x1,4x2,4x3,4x4,4x5,5x1,5x2,5x3,5x4,5x5
const,-0.029416,-0.190482,-0.099527,0.119059,-0.026688,-0.106221,-0.076151,0.020888,-0.062690,-0.067281,...,-0.121678,-0.089212,-0.042684,0.006872,-0.237798,-0.033338,-0.117663,0.014910,-0.013806,0.115886
market_excess,1.056340,0.765035,0.723490,0.961909,2.711570,0.946230,0.769014,1.103081,0.570437,0.923638,...,1.214024,1.100978,1.194372,1.282094,-0.048217,1.386433,1.146080,1.259575,1.075487,0.919919
smb,1.662196,1.069970,0.849633,1.195644,0.867727,1.256296,0.710613,0.324206,1.085800,1.078657,...,0.500109,0.167193,-0.244927,0.216487,1.321678,-1.232482,-0.611866,-0.181064,-0.027483,0.193097
hml,-0.089796,0.185565,0.158751,-0.111349,0.935892,-0.067947,0.116601,0.192836,0.660222,1.139984,...,0.152639,0.197297,0.181034,0.253083,1.735753,0.128605,0.261148,0.035656,0.176194,-0.000652


In [18]:
regression_results.to_csv('result.csv')

In [19]:
portfolio_returns.to_csv('portfolio_formation-1.5.csv')

In [20]:
factors_cleaned.to_csv('ffm_factors-1.5.csv')